In this notebook we cluster the WBAT probe measured target spectra

In [1]:
import time
import os.path
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from tenacity import retry, stop_after_attempt
import glob
import scipy as sc

from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from skimage import feature
from yellowbrick.cluster.elbow import kelbow_visualizer

import sys, errno  


In [2]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

In [3]:
palette = sns.color_palette('Paired', n_colors=16)
plt.style.use(['seaborn-colorblind'])
plt.rc('font', family='Arial')
SMALL_SIZE = 14
BIGGER_SIZE = 15
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['xtick.direction'] = 'inout'
plt.rcParams['ytick.direction'] = 'inout'
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['ytick.major.width'] = 1

savefigs_path = 'C:/Users/mbd/OneDrive - Akvaplan-niva AS/PhD-APN/ChaptersandExperiments/AFKABAN'

In [4]:
# -- USER-DETERMINED PARAMETERS -----------------------------------------------
letter_drive = 'F'
WBATpath_s = f'{letter_drive}:/AFKABAN/WBAT_data_PNC2023/StrongTS/'
WBATpath_w = f'{letter_drive}:/AFKABAN/WBAT_data_PNC2023/WeakTS/'

# Read saved df

In [5]:
WBATstrong_df = pd.read_feather(WBATpath_s+'/WBATstrong_tilt_df.feather')
WBATweak_df = pd.read_feather(WBATpath_w+'/WBATweak_tilt_df.feather')


# Cluster spectra
sort by frequency bandwith

In [7]:
def select_ts_bandwidth(df):
    'Function to seperated target spectra from 120 kHz and 200 kHz echosounders'
    ind_120 = np.where(np.isnan(df['201.129']))[0]
    ts_s_120 = np.where(df.columns.values=='96.048')[0][0] #remove 5% of spectra on either side b/c ramping
    ts_e_120 = np.where(df.columns.values=='160.565')[0][0]
    
    ind_200 = np.where(np.isnan(df['120.242']))[0]
    ts_s_200 = np.where(df.columns.values=='191.048')[0][0]
    ts_e_200 = np.where(df.columns.values=='251.532')[0][0]

    df_120 = df.iloc[ind_120,ts_s_120:ts_e_120]
    df_200 = df.iloc[ind_200,ts_s_200:ts_e_200]
    
    return df_120, df_200

In [8]:
WBATs_df_120, WBATs_df_200 = select_ts_bandwidth(WBATstrong_df)
WBATw_df_120, WBATw_df_200 = select_ts_bandwidth(WBATweak_df)


In [9]:
WBATs_df_200

,191.048,193.065,195.081,197.097,199.113,201.129,203.145,205.161,207.177,209.194,...,231.371,233.387,235.403,237.419,239.435,241.452,243.468,245.484,247.500,249.516
1,-37.660356,-38.427479,-39.453480,-40.190402,-40.631182,-40.367937,-39.522624,-40.339023,-40.881227,-41.216259,...,-49.398990,-47.626931,-46.700810,-46.549967,-46.152182,-46.208204,-45.497938,-43.557985,-41.951521,-40.987785
2,-39.328284,-39.085955,-38.988598,-38.888979,-38.911596,-38.737421,-38.927967,-40.859722,-42.050303,-42.468384,...,-44.176216,-42.959324,-42.244137,-41.838982,-41.448656,-41.825273,-42.258060,-42.906553,-43.912914,-44.046485
4,-33.820499,-34.559407,-33.273271,-32.611817,-32.913130,-33.522182,-33.890397,-33.740221,-33.097160,-32.826144,...,-36.614918,-33.979064,-32.036711,-32.743753,-34.655526,-32.281027,-31.144245,-32.080060,-34.660985,-36.076556
6,-32.061368,-31.698575,-32.003345,-32.819984,-33.106156,-32.519526,-32.465453,-33.573640,-33.722166,-33.782060,...,-32.838658,-32.917061,-33.121815,-32.828713,-32.760265,-33.891258,-34.334467,-34.211610,-33.889033,-33.689278
9,-42.604442,-43.824794,-44.675689,-44.934032,-44.798915,-44.500964,-44.129923,-44.453868,-44.439296,-44.025783,...,-42.985311,-42.756616,-42.293766,-42.204324,-41.804105,-41.004348,-40.864064,-40.898742,-41.055581,-41.377505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Select a df and run clustering

In [10]:
def kmean_spectra(df, num_clus=6):
    'Function to run kmean clusering on target spectra of a species'
    # Set up kmeans
    kmeans = KMeans(n_clusters=num_clus)
    
    # Normalize by the median of each target spectra
    df_norm = df.div(df.max(axis=1), axis=0)
    
    # Fit data
    y_kmean = kmeans.fit_predict(df_norm)

    return y_kmean, kmeans
    

In [11]:
num_clus=4

WBATs_120_ykmean, k = kmean_spectra(WBATs_df_120, num_clus=num_clus)
WBATs_200_ykmean, k = kmean_spectra(WBATs_df_200, num_clus=num_clus)

WBATw_120_ykmean, k = kmean_spectra(WBATw_df_120, num_clus=num_clus)
WBATw_200_ykmean, k = kmean_spectra(WBATw_df_200, num_clus=num_clus)

ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Add kmeans group to main dataframe
def add_ykmean(df, y_120, y_200, df_f120, df_f200):
    y_120 = pd.DataFrame(y_120, index=df_f120.index, columns=['y_kmean'])
    y_200 = pd.DataFrame(y_200+6, index=df_f200.index, columns=['y_kmean'])
    y = pd.concat([y_120,y_200])
    df['y_kmean']=y
    df['abs_tilt'] = np.abs(df['tilt'])
    return df

In [ ]:
WBATstrong_df = add_ykmean(WBATstrong_df, WBATs_120_ykmean, WBATs_200_ykmean, WBATs_df_120, WBATs_df_200)

WBATweak_df = add_ykmean(WBATweak_df, WBATw_120_ykmean, WBATw_200_ykmean, WBATw_df_120, WBATw_df_200)


### Plot kmeans clustering results

In [ ]:
df_list = [WBATs_df_120,WBATw_df_120]
y_list = [WBATs_120_ykmean, WBATw_120_ykmean]
df2_list = [WBATs_df_200, WBATw_df_200]
y2_list = [WBATs_200_ykmean, WBATw_200_ykmean]
    
name_list = ['Strong targets', 'Weak targets']
fig,ax = plt.subplots(2,1, figsize=(8,8), sharex=True)
axes = ax.flatten()
N1 = np.zeros(num_clus)
N2 = np.zeros(num_clus)

for i_df in range(len(df_list)):
    # 120 kHz
    df1 = df_list[i_df]
    y_kmean1 = y_list[i_df]
    freqs_120 = df1.columns.unique().values.astype(np.float64)

    for i_cluster in range(num_clus):
        i_ts1 = np.where(y_kmean1==i_cluster)[0]
        N1[i_cluster] = len(i_ts1)
        
    Nsort_ind = np.argsort(N1)
    i_c = 0
    for i_cluster in Nsort_ind:
        
        i_ts1 = np.where(y_kmean1==i_cluster)[0]
        axes[i_df].plot(freqs_120,np.mean(df1.iloc[i_ts1,:].T,axis=1), c=palette[i_c], label=f'Cluster {i_cluster:d} ; {N1[i_cluster]*100/len(df1):.0f}%')
        axes[i_df].fill_between(freqs_120,df1.iloc[i_ts1,:].quantile(q=0.25), df1.iloc[i_ts1,:].quantile(q=0.75), color=palette[i_c], alpha=0.1)
        i_c += 1
        
    # 200 kHz
    df2 = df2_list[i_df]
    y_kmean2 = y2_list[i_df]
    freqs_200 = df2.columns.unique().values.astype(np.float64)

    for i_cluster in range(num_clus):
        i_ts2 = np.where(y_kmean2==i_cluster)[0]
        N2[i_cluster] = len(i_ts2) 
        
    N2sort_ind = np.argsort(N2)
    i_c = 0
    for i_cluster in N2sort_ind:
        i_ts2 = np.where(y_kmean2==i_cluster)[0]
        axes[i_df].plot(freqs_200,np.mean(df2.iloc[i_ts2,:].T,axis=1), c=palette[i_c+num_clus], label=f'Cluster {i_cluster+num_clus:d} ; {N2[i_cluster]*100/len(df2):.0f}%')
        axes[i_df].fill_between(freqs_200,df2.iloc[i_ts2,:].quantile(q=0.25), df2.iloc[i_ts2,:].quantile(q=0.75), color=palette[i_c+num_clus], alpha=0.1)
        i_c +=1
        
    axes[i_df].legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2, fontsize=12, frameon=True)
    axes[i_df].set_title(f'{name_list[i_df]}', fontsize=15)
    
axes[0].set_xlabel('Frequency (kHz)', fontsize=15)
axes[0].set_ylabel('Target Strength (dB re 1m$^2$)', fontsize=13)
axes[1].set_ylabel('Target Strength (dB re 1m$^2$)', fontsize=13)

#fig.savefig('C:/Users/mbd/OneDrive - Akvaplan-niva AS/PhD-APN/PhDdocs/ConferencesMeetings/WGFAST2023/clusters.jpeg', format='jpeg', dpi = 300, facecolor='w', bbox_inches='tight')

Observations:
- Atlanctic cod has a larger (~5dB) dip in the traget spectra that shifts horizontally. What causes the shift?
- Polar cod (both experiments are similar, great start, has no consistent dips but positive, zero or negative slope.
- Pandalus does not have enough targets for clustering. Too much variability for the few amount of tracked detections. But Pandalus can be differentiated with a threshold
- clusters still have a lot of variability!

Can we use to clusters as centroids of clusters for classification? 

Note: The clusters in 120 and 200 are not related! They have been clustered independently. Each tracked SED (calibrated beam compensared TS) is unaveraged or smoothed.

# Can we correlate cluster number to tilt angle??

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(8,4), sharey=True)
axes = ax.flatten()
sns.stripplot(WBATweak_df.iloc[WBATw_df_120.index.values,:], x='y_kmean', y='tilt', ax = axes[0], size=1)
sns.stripplot(WBATweak_df.iloc[WBATw_df_200.index.values,:], x='y_kmean', y='tilt', ax = axes[1], size=1)

axes[0].set(title = 'WBAT probe weak at 120 kHz', xlabel='Cluster ID', ylabel='Tilt (degrees)')
axes[1].set(title = 'WBAT probe weak at 200 kHz', xlabel='Cluster ID', ylabel='Tilt (degrees)');

Cluster number and orientation does not seem to be related.

Similar distributions for all species, both frequency bandwidths and each cluster.

# Can we correlate cluster number to track number?? 
Do detection within the same track tend to be clustered together? If yes, it would show that broadband variability may be explained by individual fish length, material properties or other scattering properties that are relatively consistent for a single fish.

In [ ]:
count=0

In [ ]:
## Get region list for each frequency bandwidth
def select_region_bandwidth(df):
    'Function to seperated target spectra from 120 kHz and 200 kHz echosounders'
    ind_120 = np.where(np.isnan(df['200.000']))[0]
    ts_s_120 = np.where(df.columns.values=='90.000')[0][0]
    ts_e_120 = np.where(df.columns.values=='160.000')[0][0]
    
    ind_200 = np.where(np.isnan(df['120.000']))[0]
    ts_s_200 = np.where(df.columns.values=='185.000')[0][0]
    ts_e_200 = np.where(df.columns.values=='255.000')[0][0]
    
    reg_120 = df['Region_name'][ind_120] 
    reg_200 = df['Region_name'][ind_200] 
    
    return reg_120, reg_200

In [ ]:
a_reg_120, a_reg_200 = select_region_bandwidth(WBATstrong_df)
p_reg_120, p_reg_200 = select_region_bandwidth(WBATweak_df)
p2_reg_120, p2_reg_200 = select_region_bandwidth(p2_df)


In [ ]:
a_reg_120.unique()

In [ ]:
# Select dataset
df_list = [a_df_120,a_df_200,p_df_120,p_df_200,p2_df_120,p2_df_200]
y_list = [a_120_ykmean,a_200_ykmean, p_120_ykmean,p_200_ykmean, p2_120_ykmean,p2_200_ykmean]
regions_list = df_list = [a_reg_120,a_reg_200,p_reg_120,p_reg_200,p2_reg_120,p2_reg_200]

select = 5
df=df_list[select]
y_kmean = y_list[select]
regions = regions_list[select]


#---------
all_regions = regions.unique()
n_cluster_track = {}
cluster_counts ={}
n_cluster_ratio = []
n_reg = []
biggest_cluster_ratio = []

count = 0
for i_reg in all_regions:
    ind_reg = np.where(regions==i_reg)[0]
    if len(ind_reg)>2:
        y_reg = y_kmean[ind_reg]

        n_cluster_track[count], cluster_counts[count] = np.unique(y_reg,return_counts=True)
        
        biggest_cluster_ratio = np.append(biggest_cluster_ratio,max(cluster_counts[count])/len(ind_reg))

        n_cluster_ratio = np.append(n_cluster_ratio,(len(n_cluster_track[count])/len(ind_reg)))
        n_reg = np.append(n_reg, len(ind_reg))
        count += 1

In [ ]:
fig,ax = plt.subplots(2,2)
axes = ax.flatten()
axes[0].hist(n_cluster_ratio)

axes[1].plot(n_reg,n_cluster_ratio, '.')
axes[1].set(xlabel='Number of detections in track',ylabel='Consistency');

axes[2].hist(biggest_cluster_ratio)

axes[3].plot(n_reg,n_cluster_ratio*n_reg, '.')

#### How many tracks contain one cluster?

In [ ]:
for i in range(len(n_cluster_track)):


In [ ]:
n_cluster_track[1]

Haven't proved it yet but it really doesnt seem to be a driving factor

## Try pca? in Python

In [ ]:
le = LabelEncoder() # Maps labels -> int (e.g. Atlantic cod -> 0, Polar cod -> 1)
a_df['RegionID'] = le.fit_transform(a_df.Region_name)

In [ ]:
sub_df = np.array(a_df[['tilt','RegionID','y_kmean','abs_tilt']])

In [ ]:
std_scaler = StandardScaler()
 
df_scaled = std_scaler.fit_transform(sub_df['tilt'])
df_scaled = pd.DataFrame(df_scaled, columns=[
  'tilt','RegionID','y_kmean','abs_tilt'])
 

In [ ]:
X = df_scaled[~np.any(np.isnan(sub_df),axis=1)]

In [ ]:
X

In [ ]:
pca = PCA(n_components=4)

In [ ]:
pca.fit(X)

In [ ]:
rng = np.random.RandomState(0)
n_samples = len(X)
pca = PCA(n_components=4).fit(X)


plt.scatter(X.iloc[:, 0], X.iloc[:, 1], alpha=0.3, label="samples")
for i, (comp, var) in enumerate(zip(pca.components_, pca.explained_variance_)):
    comp = comp * var  # scale component by its variance explanation power
    plt.plot(
        [0, comp[0]],
        [0, comp[1]],
        label=f"Component {X.columns[i]}",
        linewidth=5,
        color=f"C{i + 2}",
    )
plt.gca().set(
    aspect="equal",
    title="2-dimensional dataset with principal components",
    xlabel="first feature",
    ylabel="second feature",
)
plt.legend()
plt.show()

# Storage Area

### Elbow algorithm to find optimal clusters

Run elbow algorithm for optimal number of clusters.

Optimal value is found at the "elbow" of the distortion results.

In [ ]:
#Use the quick method and immediately show the figure
df = WBATs_df_120
df_norm = df.div(df.median(axis=1), axis=0)
kelbow_visualizer(KMeans(), df_norm, k=(1,20))